# Assignment #1: Building an inverted index

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf value
* Write a short report of 1 to 2 pages on the assignment
* Read a short text on an industrial system

## Description

### Outline

Conceptually, an index consists of rows with one word per row and and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

### Preparation

<p>You will create an index for a corpus of Selma Lagerlöf's works:</p>
<p>Download the <a href="https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip">Selma
                folder</a> and uncompress it. It contains novels by <a href="https://sv.wikipedia.org/wiki/Selma_Lagerl%C3%B6f">
                Selma Lagerlöf</a>. The text of these novels was extracted
                from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
                <a href="https://litteraturbanken.se/">Litteraturbanken</a>.
            </p>

In [7]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/selmanovels/kejsaren.txt
/kaggle/input/selmanovels/osynliga.txt
/kaggle/input/selmanovels/jerusalem.txt
/kaggle/input/selmanovels/marbacka.txt
/kaggle/input/selmanovels/nils.txt
/kaggle/input/selmanovels/troll.txt
/kaggle/input/selmanovels/bannlyst.txt
/kaggle/input/selmanovels/herrgard.txt
/kaggle/input/selmanovels/gosta.txt


### Running the Indexer

Your final program will take a corpus as input (here the Selma novels) and produce an index of all the words with their positions. you should be able to run it this way:
<pre>$ python indexer.py folder_name</pre>
In this lab, you will write it in a Jupyter Notebook. The conversion into a Python program is left as an optional exercise.

## Programming the Indexer

To make programming easier, you will split this exercise into five steps:
1. Index one file;
2. Read the content of a folder
3. Create a master index for all the files
4. Use tfidf to represent the documents (novels)
5. Compare the documents of a collection

You will use dictionaries to represent the postings.

### Indexing one file

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, you will use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use <tt>finditer()</tt> to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

<p>Below is an excerpt of the index of the bannlyst text for the words <i>gjord</i>, <i>uppklarnande</i>, and
            <i>stjärnor</i>. The data is stored in a dictionary:
        </p>
        <pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
        </pre>
        <p>The word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at
            position 8617, and <i>stjärnor</i>, once at position 8641.
        </p>

#### Imports

Some imports you could need. Add others as needed

In [8]:
import regex as re
import pickle
import sys
import math
import time

#### Writing a tokenizer 

Write a Unicode regular expression to find words defined as sequences of letters.

In [9]:
regex = '\p{L}+'

In [10]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write a function to tokenize a text. Return their positions.

In [11]:
def tokenize(text):
    return re.finditer(regex, text)

In [12]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

[<regex.Match object; span=(0, 2), match='En'>,
 <regex.Match object; span=(3, 7), match='gång'>,
 <regex.Match object; span=(8, 12), match='hade'>,
 <regex.Match object; span=(13, 15), match='de'>,
 <regex.Match object; span=(16, 18), match='på'>,
 <regex.Match object; span=(19, 27), match='Mårbacka'>,
 <regex.Match object; span=(28, 30), match='en'>,
 <regex.Match object; span=(31, 39), match='barnpiga'>,
 <regex.Match object; span=(41, 44), match='som'>,
 <regex.Match object; span=(45, 50), match='hette'>,
 <regex.Match object; span=(51, 55), match='Back'>,
 <regex.Match object; span=(56, 61), match='Kajsa'>]

#### Extracting indices

Write a function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the keys a list of positions.

In [13]:
def text_to_idx(tokens):
    dict = {}
    for token in tokens:
        if token.group() in dict:
            dict[token.group()].append(token.start())
        else:
            dict[token.group()] = [token.start()]
    return dict
 

#   positions = []
#    matches = re.finditer(regex, tokens)
#    for match in matches:
#        positions.append(match.start())
#    return positions


In [14]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 28],
 'gång': [3],
 'hade': [8],
 'de': [13],
 'på': [16],
 'mårbacka': [19],
 'barnpiga': [31],
 'som': [41],
 'hette': [45],
 'back': [51],
 'kajsa': [56]}

#### Reading one file

Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [15]:
file = open('/kaggle/input/selmanovels/marbacka.txt', 'r', encoding='utf-8')
text_f = file.read().lower().strip()
mb_tokens = tokenize(text_f)
idx = text_to_idx(mb_tokens)

In [16]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

#### Saving the index

Save your index in a file so that you can reuse it. Use the pickel module.

In [17]:
pickle.dump( idx, open( "idx.p", "wb" ) )

In [18]:
idx = pickle.load( open( "idx.p", "rb" ) )

In [19]:
idx['mårbacka']

[16,
 139,
 752,
 1700,
 2582,
 3324,
 15117,
 15404,
 27794,
 42175,
 49126,
 50407,
 52053,
 60144,
 63374,
 64910,
 67182,
 67330,
 67799,
 67824,
 69232,
 71328,
 72099,
 74147,
 74255,
 74614,
 76610,
 76884,
 77138,
 77509,
 77787,
 77936,
 78574,
 80597,
 81782,
 82003,
 84363,
 84786,
 85251,
 89837,
 97093,
 98642,
 100474,
 105063,
 105298,
 105721,
 108710,
 109133,
 112844,
 113725,
 114997,
 115583,
 115833,
 116368,
 116557,
 121896,
 124823,
 126409,
 126542,
 128758,
 130976,
 131939,
 132826,
 136914,
 137187,
 137872,
 139196,
 140721,
 142324,
 146781,
 151497,
 154335,
 155139,
 155438,
 155886,
 156405,
 158108,
 159817,
 160107,
 161158,
 162085,
 165847,
 168316,
 168528,
 169111,
 170333,
 172684,
 182047,
 182427,
 186362,
 189535,
 190999,
 191110,
 193177,
 196686,
 202552,
 206340,
 207789,
 208382,
 209874,
 210525,
 217464,
 219933,
 221393,
 221533,
 221880,
 222213,
 224190,
 229501,
 229598,
 230783,
 231453,
 232140,
 234427,
 236193,
 236950,
 240168,

### Reading the content of a folder

<p>Write a function that reads all the files in a folder with a specific suffix (txt). You will need the Python
            os package,
            see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You
            will return the file names in a list.
        </p>

You can reuse this function:

In [20]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

In [21]:
corpus_files = get_files("/kaggle/input/selmanovels", 'txt')
corpus_files

['kejsaren.txt',
 'osynliga.txt',
 'jerusalem.txt',
 'marbacka.txt',
 'nils.txt',
 'troll.txt',
 'bannlyst.txt',
 'herrgard.txt',
 'gosta.txt']

### Creating a master index

<p>Complete your program with the creation of master index, where you will associate each word
            of the corpus with the files, where it occur and its positions. (a posting list)
        </p>
        <p>Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:
        </p>

In [22]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

{'samlar': {'troll.txt': [641880, 654233],
  'nils.txt': [51805, 118943],
  'osynliga.txt': [399121],
  'gosta.txt': [313784, 409998, 538165]},
 'ände': {'troll.txt': [39562, 650112],
  'kejsaren.txt': [50171],
  'marbacka.txt': [370324],
  'nils.txt': [1794],
  'osynliga.txt': [272144]}}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

In [23]:
def get_master_idx(dir, suffix):
    master_dict = {}
    for file_name in get_files(dir, suffix):
        
        # Read file
        rfile = open(dir + "/" + file_name, 'r', encoding='utf-8')
        file = rfile.read().lower().strip()
        
        # Tokenize and return dict with idx.
        tokens = tokenize(file)
        idx = text_to_idx(tokens)
        
        # Add to master idx.
        for key in idx.keys():
            if key in master_dict:
                if file_name in master_dict[key]:
                    master_dict[key][file_name].append(idx[key])
                else:
                    master_dict[key][file_name] = idx[key]
            else:
                dict = {}
                dict[file_name] = idx[key]
                master_dict[key] = dict

    return master_dict

master_index = get_master_idx("/kaggle/input/selmanovels", 'txt')
master_index['nils']

{'kejsaren.txt': [263335, 266581],
 'jerusalem.txt': [171205, 324540, 324586],
 'marbacka.txt': [347421, 365256],
 'nils.txt': [17,
  3629,
  18122,
  18159,
  49682,
  61218,
  99050,
  99446,
  100632,
  100850,
  107420,
  122516,
  138978,
  159346,
  187665,
  204863,
  205377,
  215613,
  281268,
  285357,
  285454,
  292483,
  292713,
  292980,
  294817,
  303588,
  308340,
  317519,
  333659,
  361086,
  405967,
  406139,
  406724,
  414324,
  459738,
  503209,
  563342,
  581222,
  602124,
  626011,
  636848,
  667690,
  667951,
  674565,
  682702,
  695641,
  701899,
  702256,
  704143,
  707989,
  759870,
  779155,
  866429,
  866532,
  867887,
  868874,
  874170,
  938003,
  992433,
  992805,
  1011668,
  1019630,
  1019962,
  1020373,
  1045606,
  1063798,
  1064583,
  1065559,
  1068924,
  1079624,
  1079782,
  1080261,
  1082400,
  1083201,
  1083350,
  1085516,
  1086605,
  1086798,
  1091380],
 'troll.txt': [273705]}

In [24]:
master_index['samlar']

{'osynliga.txt': [399121],
 'nils.txt': [51805, 118943],
 'troll.txt': [641880, 654233],
 'gosta.txt': [313784, 409998, 538165]}

In [25]:
master_index['mårbacka']

{'marbacka.txt': [16,
  139,
  752,
  1700,
  2582,
  3324,
  15117,
  15404,
  27794,
  42175,
  49126,
  50407,
  52053,
  60144,
  63374,
  64910,
  67182,
  67330,
  67799,
  67824,
  69232,
  71328,
  72099,
  74147,
  74255,
  74614,
  76610,
  76884,
  77138,
  77509,
  77787,
  77936,
  78574,
  80597,
  81782,
  82003,
  84363,
  84786,
  85251,
  89837,
  97093,
  98642,
  100474,
  105063,
  105298,
  105721,
  108710,
  109133,
  112844,
  113725,
  114997,
  115583,
  115833,
  116368,
  116557,
  121896,
  124823,
  126409,
  126542,
  128758,
  130976,
  131939,
  132826,
  136914,
  137187,
  137872,
  139196,
  140721,
  142324,
  146781,
  151497,
  154335,
  155139,
  155438,
  155886,
  156405,
  158108,
  159817,
  160107,
  161158,
  162085,
  165847,
  168316,
  168528,
  169111,
  170333,
  172684,
  182047,
  182427,
  186362,
  189535,
  190999,
  191110,
  193177,
  196686,
  202552,
  206340,
  207789,
  208382,
  209874,
  210525,
  217464,
  219933,
  2213

Save your master index in a file and read it again

In [26]:
pickle.dump(master_index, open( "master_idx.p", "wb" ) )
master_index = pickle.load( open( "master_idx.p", "rb" ) )

In [27]:
master_index['samlar']

{'osynliga.txt': [399121],
 'nils.txt': [51805, 118943],
 'troll.txt': [641880, 654233],
 'gosta.txt': [313784, 409998, 538165]}

#### Concordances

Write a function to extract the concordances of a word within a window of `window` characters

In [28]:
def concordance(text, indexer, corpus_files, width):
    concordances = []
    
    pattern = text
    # spaces match tabs and newlines
    pattern = re.sub(' ', '\\s+', pattern)
    # Replaces newlines with spaces in the text
    text = re.sub('\s+', ' ', text)
    concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
                   .format(pattern=pattern, width=width))
    for file_name in corpus_files:
        rfile = open("/kaggle/input/selmanovels" + "/" + file_name, 'r', encoding='utf-8')
        file = rfile.read().lower().strip()
        for match in re.finditer(concordance, file):
            concordances.append(match.group(1))
    return concordances

In [29]:
concordance('samlar', master_index, corpus_files, 25)

[' till höger i kärran och samlar just ihop tömmarna, och ',
 'som man i andra länder insamlar bär och frukter, så går ',
 ' bara, att du i all hast samlar ihop så mycket boss och ',
 'ar stannat hemma, och nu samlar de sig för att intränga ',
 'en örtkunnig läkare, som samlar in markens växter för at',
 'on sålunda handlar och insamlar för de i fiendehänder fa',
 'älper dem, och medan hon samlar och handlar för deras rä',
 'om ligger nära borg, och samlar ihop ett litet middagssä',
 'men hon samlar upp allt detta som glöda',
 'därmed samlar han korten tillhopa, res']

### Representing Documents with tf-idf

<p>Once you have created the index, you will represent each document in your corpus as a word vector.
            You will define the value of a word in a document with the tf-idf
            metric. Tf will be the relative frequency of the term in the document and idf, the logarithm base 10 of the
            inverse document
            frequency.
        </p>
        <p>You have below the tf-idf values for a few words. In our example, the word <i>gås</i> has the value
            0 in bannlyst.txt and the value 0.000101001964 in nils.txt
        </p>
        <pre>
troll.txt
	känna	 0.0
	gås	 0.0
	nils	 2.148161748868631e-06
	et	 0.0
kejsaren.txt
	känna	 0.0
	gås	 0.0
	nils	 8.08284798629935e-06
	et	 8.273225429362848e-05
marbacka.txt
	känna	 0.0
	gås	 0.0
	nils	 7.582276564686669e-06
	et	 9.70107989686256e-06
herrgard.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
nils.txt
	känna	 0.0
	gås	 0.00010100196417506702
	nils	 0.00010164426900380124
	et	 0.0
osynliga.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
jerusalem.txt
	känna	 0.0
	gås	 0.0
	nils	 4.968292117670952e-06
	et	 0.0
bannlyst.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
gosta.txt
	känna	 0.0
	gås	 0.0
	nils	 0.0
	et	 0.0
        </pre>


In [30]:
def read_files(dir):
    files = {}
    for file_name in corpus_files:
        rfile = open(dir + "/" + file_name, 'r', encoding='utf-8')
        file = rfile.read().lower().strip()
        files[file_name] = file
    return files
        
N = len(corpus_files)

def get_word_vec():
    tfidf = {}
    #tfidf = dict.fromkeys(corpus_files, 0)
    files = read_files("/kaggle/input/selmanovels")
    
    for document in list(files.keys()):
        document_words = {}
        tf = 0.0
        idf = 0.0
        for word in master_index.keys():
            d = len(master_index[word].keys())
            
            if document in master_index[word]:
                tf = len(master_index[word][document]) / len(files[document])
                idf = math.log(N/d, 10)
                document_words[word] = tf*idf
            else:
                document_words[word] = 0.0
        tfidf[document] = document_words
    
    return tfidf

tfidf = get_word_vec()

In [31]:
tfidf['troll.txt']['känna']

0.0

In [32]:
tfidf['troll.txt']['nils']

3.84249211027629e-07

### Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tfidf representation and present
            your results in a matrix. You will include this matrix in your report.

#### Cosine similarity

Write a function computing the cosine similarity between two documents

In [33]:
from numpy import dot
from numpy.linalg import norm


def cosine_similarity(doc1, doc2):
    lv_doc1 = list(doc1.values())
    lv_doc2 = list(doc2.values())
    return dot(lv_doc1, lv_doc2)/(norm(lv_doc1)*norm(lv_doc2))


#### Similarity matrix

Compute the similarity matrix between the documents of the corpus

In [34]:
matrix = []
max_similarity = 0
most_sim_f1 = ''
most_sim_f2 = ''
for document_one in tfidf.keys():
    row = []
    for document_two in tfidf.keys():
        if document_one != document_two:
            sim = cosine_similarity(tfidf[document_one], tfidf[document_two])
            if sim > max_similarity:
                max_similarity = sim
                most_sim_f1 = document_one
                most_sim_f2 = document_two
        else:
            sim = 0
        row.append(sim)
            
    matrix.append(row)
    
from pandas import DataFrame as df
func = lambda x: round(x,3)

matrix = [list(map(func, i)) for i in matrix]
df_m = df(matrix)
df_m.columns = list(tfidf.keys())
df_m

,kejsaren.txt,osynliga.txt,jerusalem.txt,marbacka.txt,nils.txt,troll.txt,bannlyst.txt,herrgard.txt,gosta.txt
0,0.000,0.051,0.002,0.071,0.050,0.181,0.024,0.001,0.048
1,0.051,0.000,0.028,0.093,0.111,0.193,0.052,0.005,0.125
2,0.002,0.028,0.000,0.005,0.005,0.007,0.006,0.371,0.004
3,0.071,0.093,0.005,0.000,0.085,0.147,0.037,0.004,0.080
4,0.050,0.111,0.005,0.085,0.000,0.188,0.051,0.005,0.105
5,0.181,0.193,0.007,0.147,0.188,0.000,0.089,0.004,0.196
6,0.024,0.052,0.006,0.037,0.051,0.089,0.000,0.001,0.049
7,0.001,0.005,0.371,0.004,0.005,0.004,0.001,0.000,0.003
8,0.048,0.125,0.004,0.080,0.105,0.196,0.049,0.003,0.000


Give the name of the two novels that are the most similar.

In [35]:
print("Most similar:", most_sim_f1, most_sim_f2, "Similarity:", max_similarity)

Most similar: jerusalem.txt herrgard.txt Similarity: 0.37068942387339526


<h2>Reading</h2>

<p>Read the text:
            <i>Challenges in Building Large-Scale Information Retrieval Systems</i>
            about the history of
            <a href="https://research.google.com/people/jeff/WSDM09-keynote.pdf">Google indexing</a>
            by
            <a href="https://research.google.com/pubs/jeff.html">Jeff Dean</a>.
            In your report, tell how your index encoding is related to what Google did.
            You must identify the slide where you have the most similar indexing technique and write the
            slide number in your report.
        </p>